# 翻轉醫療爬蟲

https://flippingmed.com/

首先我們觀察flippingmed這個網站，

發現他換頁的邏輯是在一個post的api裡面 (page)，

我們把post的這個request dump出來，

寫一個while loop，每次loop就把這個page + 1

我們另外把每頁的return資料比數調高到50 (query_args[posts_per_page])

In [3]:
import requests
import json
import urllib.parse

url = "https://flippingmed.com/?infinity=scrolling"
data = {'action': 'infinite_scroll',
 'currentday': '29.05.18',
 'last_post_date': '2018-06-04+12:13:30',
 'order': 'DESC',
 'page': 1, # 這是我們要換頁的變數，把他設定成一個int
 'query_args[attachment]': '',
 'query_args[attachment_id]': '0',
 'query_args[author]': '',
 'query_args[author_name]': '',
 'query_args[cache_results]': 'false',
 'query_args[cat]': '',
 'query_args[category_name]': '',
 'query_args[comments_per_page]': '50',
 'query_args[day]': '0',
 'query_args[embed]': '',
 'query_args[error]': '',
 'query_args[feed]': '',
 'query_args[fields]': '',
 'query_args[hour]': '',
 'query_args[ignore_sticky_posts]': 'false',
 'query_args[lazy_load_term_meta]': 'true',
 'query_args[m]': '',
 'query_args[menu_order]': '',
 'query_args[meta_key]': '',
 'query_args[meta_value]': '',
 'query_args[minute]': '',
 'query_args[monthnum]': '0',
 'query_args[name]': '',
 'query_args[no_found_rows]': 'false',
 'query_args[nopaging]': 'false',
 'query_args[order]': 'DESC',
 'query_args[p]': '0',
 'query_args[page_id]': '0',
 'query_args[paged]': '0',
 'query_args[pagename]': '',
 'query_args[post_parent]': '',
 'query_args[post_type]': '',
 'query_args[posts_per_page]': '50',
 'query_args[preview]': '',
 'query_args[s]': '',
 'query_args[second]': '',
 'query_args[sentence]': '',
 'query_args[static]': '',
 'query_args[subpost]': '',
 'query_args[subpost_id]': '',
 'query_args[suppress_filters]': 'false',
 'query_args[tag]': '',
 'query_args[tag_id]': '',
 'query_args[tb]': '',
 'query_args[title]': '',
 'query_args[update_post_meta_cache]': 'true',
 'query_args[update_post_term_cache]': 'true',
 'query_args[w]': '0',
 'query_args[year]': '0',
 'query_before': '2018-06-05+11:04:29',
 'scripts[]': 'aepc-pixel-events',
 'styles[]': 'jetpack_css'}
headers = {'accept': '*/*',
 'accept-encoding': 'gzip, deflate, br',
 'accept-language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7',
 'content-length': '2835',
 'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
 'cookie': 'tk_or=%22https%3A%2F%2Fwww.google.com.tw%2F%22; tk_r3d=%22https%3A%2F%2Fwww.google.com.tw%2F%22; __utma=73359812.1630321809.1528167876.1528167876.1528167876.1; __utmc=73359812; __utmz=73359812.1528167876.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmt=1; tk_lr=%22%22; __utmb=73359812.7.10.1528167876',
 'origin': 'https://flippingmed.com',
 'referer': 'https://flippingmed.com/',
 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.62 Safari/537.36',
 'x-requested-with': 'XMLHttpRequest'}

article_links = []
while True:
    print("[INFO] crawling page %s"%data['page'])
    
    # 送出post request
    resp = requests.post(url, data=data, headers=headers)
    
    # 將response做json decode
    resp_body = resp.json()
    
    # 判斷是否爬到底了，以這個網站來說，爬到底的時候會回傳一個json裡面的資料是{"type": "empty"}
    if resp_body.get('type', '') == 'empty':
        break # 爬到底就break出去這個while loop
        
    # 將resp的json取出postflair這個key，裡面再出去keys()並且cast成list type
    # 最後做URL decode的動作，把url換成中文
    links = [urllib.parse.unquote(url) for url in list(resp_body.get('postflair').keys())]
    
    # 將url list extend 進去我們的result list
    article_links.extend(links)
    
    # 將page + 1
    data['page'] += 1

[INFO] crawling page 1
[INFO] crawling page 2


KeyboardInterrupt: 

In [2]:
import os

# 建立資料夾， 如同linux的mkdir -p
os.makedirs('/tmp/mydata/', exist_ok=True)

# for loop 剛才爬下來的url list
for url in article_links:
    print(url)
    resp = requests.get(url)
    
    # 製作一個filename safe的檔名，使用url來做，把/置換成_
    filename = url.replace('/', '_')
    
    # 寫出去
    with open('/tmp/mydata/%s.html'%filename, 'w') as f:
        f.write(resp.text)

https://flippingmed.com/2016/06/22/為什麼輔具經營是進入長照產業最好的敲門磚！/
https://flippingmed.com/2018/06/04/心智食療全面預防-地中海飲食讓熟齡不失智/
https://flippingmed.com/2018/06/04/舞動幸福熟齡：雲門律動跳出活力青春/
https://flippingmed.com/2018/06/04/益智活腦＋豐富活動：有效擊敗失智全攻略/
https://flippingmed.com/2018/06/04/接軌長照與醫療-落實全人照護理念：居家醫療照/
https://flippingmed.com/2018/06/02/保健食品-肝臟疾病需注意的保健食品/
https://flippingmed.com/2018/05/31/一休和尚有說過：想站起來，要先低頭！/
https://flippingmed.com/2018/05/30/排泄達人這麼說：掌握四大重點就能找到適合的替/
https://flippingmed.com/2018/05/30/正確了解紙尿褲的構造及材質，才能真正擺脫外漏/
https://flippingmed.com/2018/05/30/了解排尿障礙的類型-才能找到對應的解決對策/
https://flippingmed.com/2018/05/30/找到紙尿褲外漏根本原因，才能徹底泌尿解決問題/


KeyboardInterrupt: 